# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# read the csv made from the weatherpy homework
city_data_df = pd.read_csv("output_resource/city_data.csv")
city_data_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Time Zone
0,0,Lata,40.16,-8.33,73.99,75,62,5.01,PT,3600
1,1,Puerto Ayora,-0.74,-90.35,70.00,77,87,10.00,EC,-21600
2,2,Saint Paul Harbor,57.79,-152.41,51.80,81,90,10.29,US,-28800
3,3,Presidencia Roque Sáenz Peña,-26.79,-60.44,81.99,17,0,6.98,AR,-10800
4,4,Bathsheba,13.22,-59.52,89.60,66,20,11.41,BB,-14400
...,...,...,...,...,...,...,...,...,...,...
702,702,Jalu,29.03,21.55,80.24,33,0,6.60,LY,7200
703,703,Wamba,8.93,8.60,76.53,81,27,2.30,NG,3600
704,704,Beringovskiy,63.05,179.32,39.63,71,10,7.23,RU,43200
705,705,Oranjemund,-28.55,16.43,67.68,35,0,21.97,NaN,7200


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# configure gmaps
gmaps.configure(api_key=g_key)

In [15]:
# add heatmap layer using humidity for the cities found in weatherpy
locations = city_data_df[["Lat", "Lng"]].astype(float)
humidity = city_data_df["Humidity"].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
# narrowing down cities for the perfect vacation destination
# temp in between 70-80 (exclusive), with zero cloudiness for the beautiful sky
vacation_city_df = city_data_df.loc[city_data_df["Max Temp"] < 80, :]
vacation_city_df = vacation_city_df.loc[vacation_city_df["Max Temp"] > 70, :]
vacation_city_df = vacation_city_df.loc[vacation_city_df["Cloudiness"] == 0, :]
vacation_city_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Time Zone
75,75,Kāzerūn,29.62,51.65,76.78,21,0,3.94,IR,12600
265,265,San Quintín,30.48,-115.95,79.97,50,0,11.52,MX,-25200
272,272,Shiraz,29.60,52.54,77.00,12,0,4.70,IR,12600
302,302,Luwingu,-10.26,29.93,72.54,23,0,7.65,ZM,7200
453,453,Abu Samrah,35.30,37.18,73.13,60,0,9.10,SY,10800
555,555,Poum,-20.23,164.02,71.24,76,0,11.03,NC,39600
610,610,Mpika,-11.83,31.45,70.12,32,0,10.83,ZM,7200
634,634,Bara,10.37,10.73,78.64,70,0,1.61,NG,3600


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [43]:
# putting the narrowed down cities into a new dataframe with selected columns
hotel_df = pd.DataFrame(vacation_city_df, columns=["City", "Country", "Lat", "Lng", "Max Temp"])
hotel_df

,City,Country,Lat,Lng,Max Temp
75,Kāzerūn,IR,29.62,51.65,76.78
265,San Quintín,MX,30.48,-115.95,79.97
272,Shiraz,IR,29.60,52.54,77.00
302,Luwingu,ZM,-10.26,29.93,72.54
453,Abu Samrah,SY,35.30,37.18,73.13
555,Poum,NC,-20.23,164.02,71.24
610,Mpika,ZM,-11.83,31.45,70.12
634,Bara,NG,10.37,10.73,78.64


In [46]:
# make a new list hotels to store nearby hotel
hotels = []
lat = hotel_df["Lat"].tolist()
lng = hotel_df["Lng"].tolist()

# run through the google api to find nearest hotel within 5000m radius
for x in range(len(hotel_df)):
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {"location": f"{lat[x]}, {lng[x]}",
             "radius": 5000,
             "types": "hotel",
             "key": g_key}
    
    request = requests.get(base_url, params=params).json()
    hotels.append(request["results"][0]["name"])

# store to hotel dataframe
hotel_df["Hotel Name"] = hotels
hotel_df

,City,Country,Lat,Lng,Max Temp,Hotel Name
75,Kāzerūn,IR,29.62,51.65,76.78,Kazerun
265,San Quintín,MX,30.48,-115.95,79.97,Lázaro Cárdenas
272,Shiraz,IR,29.60,52.54,77.00,Shiraz
302,Luwingu,ZM,-10.26,29.93,72.54,Luwingu
453,Abu Samrah,SY,35.30,37.18,73.13,Suruj
555,Poum,NC,-20.23,164.02,71.24,Poum
610,Mpika,ZM,-11.83,31.45,70.12,Mpika
634,Bara,NG,10.37,10.73,78.64,Bara


In [47]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [49]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations)
fig.add_layer(marker)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))